# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [34]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [35]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_text_ids = [[source_vocab_to_int.get(word) for word in sentence.split()] for sentence in source_text.split('\n')]
    target_text_ids = [[target_vocab_to_int.get(word) for word in sentence.split()]+ [target_vocab_to_int['<EOS>']] for sentence in target_text.split('\n')]
    #print(source_text_ids)
    return [source_text_ids,target_text_ids]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [36]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [39]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32,[None,None],name='input')
    targets_ = tf.placeholder(tf.int32,[None,None],name='targets')
    lr = tf.placeholder(tf.float32,name='learning_rate')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    
    tar_seq_len = tf.placeholder(tf.int32,[None],name='target_sequence_length')
    max_tar_seq_len = input_ = tf.reduce_max(tar_seq_len,name='max_target_len')
    source_seq_len = tf.placeholder(tf.int32,[None],name='source_sequence_length')
    return [input,targets_,lr,keep_prob,tar_seq_len,max_tar_seq_len,source_seq_len]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [40]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    rm_ending = tf.strided_slice(target_data,[0,0],[batch_size,-1],[1,1])
    dec_inp = tf.concat([tf.fill([batch_size,1],target_vocab_to_int['<GO>']),rm_ending],1)
    return dec_inp

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [41]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    encode = tf.contrib.layers.embed_sequence(rnn_inputs,source_vocab_size,encoding_embedding_size)
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell,output_keep_prob=keep_prob)
        return enc_cell
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    output,final_state = tf.nn.dynamic_rnn(enc_cell,encode,sequence_length=source_sequence_length,dtype=tf.float32)
    
    return output,final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [42]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,sequence_length=target_sequence_length)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell,output_keep_prob=keep_prob)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,helper,encoder_state,output_layer)
    dec_out,final_state,fin_seq_len = tf.contrib.seq2seq.dynamic_decode(decoder,impute_finished=True,maximum_iterations=max_summary_length)
    return dec_out



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [43]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    #print(start_of_sequence_id)
    start_tokens = tf.tile(tf.constant([start_of_sequence_id],dtype=tf.int32),[batch_size],name='start_tokens')
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,start_tokens,end_of_sequence_id)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell,output_keep_prob=keep_prob)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,helper,encoder_state,output_layer)
    dec_out,final_state,fin_seq_len = tf.contrib.seq2seq.dynamic_decode(decoder,impute_finished=True,maximum_iterations=max_target_sequence_length)
    return dec_out



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [44]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size,decoding_embedding_size],minval=-1,maxval=1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings,dec_input)
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell,output_keep_prob=keep_prob)
        return enc_cell
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    output_layer = Dense(target_vocab_size,kernel_initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    with tf.variable_scope('decode'):
        bdo_train = decoding_layer_train(encoder_state, enc_cell, dec_embed_input, target_sequence_length, 
                                     max_target_sequence_length, output_layer, keep_prob)
    
    with tf.variable_scope('decode',reuse=True):
        bdo_infer = decoding_layer_infer(encoder_state, enc_cell, dec_embeddings, target_vocab_to_int['<GO>'], 
                                         target_vocab_to_int['<EOS>'], max_target_sequence_length, 
                                         target_vocab_size, output_layer, batch_size, keep_prob)
    
    return bdo_train,bdo_infer



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [45]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    _,encoder_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    bdo_train, bdo_infer = decoding_layer(dec_input, encoder_state, target_sequence_length, 
                                          max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, 
                                          target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    return bdo_train, bdo_infer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [46]:
# Number of Epochs
epochs = 3
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 10
decoding_embedding_size = 10
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.7
display_step = 1

### Build the Graph
Build the graph using the neural network you implemented.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [49]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/538 - Train Accuracy: 0.2320, Validation Accuracy: 0.3164, Loss: 5.4034
Epoch   0 Batch    2/538 - Train Accuracy: 0.2873, Validation Accuracy: 0.3466, Loss: 4.5673
Epoch   0 Batch    3/538 - Train Accuracy: 0.0934, Validation Accuracy: 0.1211, Loss: 4.3234
Epoch   0 Batch    4/538 - Train Accuracy: 0.2785, Validation Accuracy: 0.3498, Loss: 4.3325
Epoch   0 Batch    5/538 - Train Accuracy: 0.3309, Validation Accuracy: 0.3807, Loss: 3.7660
Epoch   0 Batch    6/538 - Train Accuracy: 0.3397, Validation Accuracy: 0.3817, Loss: 3.6792
Epoch   0 Batch    7/538 - Train Accuracy: 0.3123, Validation Accuracy: 0.3771, Loss: 3.6738
Epoch   0 Batch    8/538 - Train Accuracy: 0.3182, Validation Accuracy: 0.3816, Loss: 3.5601
Epoch   0 Batch    9/538 - Train Accuracy: 0.3318, Validation Accuracy: 0.3935, Loss: 3.4848
Epoch   0 Batch   10/538 - Train Accuracy: 0.2545, Validation Accuracy: 0.3232, Loss: 3.4731
Epoch   0 Batch   11/538 - Train Accuracy: 0.3344, Validation Accuracy

Epoch   0 Batch   90/538 - Train Accuracy: 0.4554, Validation Accuracy: 0.4865, Loss: 1.6121
Epoch   0 Batch   91/538 - Train Accuracy: 0.4266, Validation Accuracy: 0.4860, Loss: 1.6759
Epoch   0 Batch   92/538 - Train Accuracy: 0.4363, Validation Accuracy: 0.4872, Loss: 1.6299
Epoch   0 Batch   93/538 - Train Accuracy: 0.4156, Validation Accuracy: 0.4666, Loss: 1.6463
Epoch   0 Batch   94/538 - Train Accuracy: 0.4047, Validation Accuracy: 0.4613, Loss: 1.6345
Epoch   0 Batch   95/538 - Train Accuracy: 0.4879, Validation Accuracy: 0.4956, Loss: 1.5000
Epoch   0 Batch   96/538 - Train Accuracy: 0.4784, Validation Accuracy: 0.5087, Loss: 1.5113
Epoch   0 Batch   97/538 - Train Accuracy: 0.4252, Validation Accuracy: 0.4936, Loss: 1.5975
Epoch   0 Batch   98/538 - Train Accuracy: 0.4632, Validation Accuracy: 0.4846, Loss: 1.4936
Epoch   0 Batch   99/538 - Train Accuracy: 0.4430, Validation Accuracy: 0.5048, Loss: 1.6220
Epoch   0 Batch  100/538 - Train Accuracy: 0.4516, Validation Accuracy

Epoch   0 Batch  179/538 - Train Accuracy: 0.4818, Validation Accuracy: 0.5265, Loss: 1.0956
Epoch   0 Batch  180/538 - Train Accuracy: 0.5032, Validation Accuracy: 0.5305, Loss: 1.0938
Epoch   0 Batch  181/538 - Train Accuracy: 0.4695, Validation Accuracy: 0.5284, Loss: 1.1567
Epoch   0 Batch  182/538 - Train Accuracy: 0.4586, Validation Accuracy: 0.5321, Loss: 1.1212
Epoch   0 Batch  183/538 - Train Accuracy: 0.5240, Validation Accuracy: 0.5300, Loss: 1.0279
Epoch   0 Batch  184/538 - Train Accuracy: 0.4926, Validation Accuracy: 0.5314, Loss: 1.0345
Epoch   0 Batch  185/538 - Train Accuracy: 0.4713, Validation Accuracy: 0.5412, Loss: 1.1027
Epoch   0 Batch  186/538 - Train Accuracy: 0.4935, Validation Accuracy: 0.5309, Loss: 1.0683
Epoch   0 Batch  187/538 - Train Accuracy: 0.5004, Validation Accuracy: 0.4984, Loss: 1.0069
Epoch   0 Batch  188/538 - Train Accuracy: 0.4891, Validation Accuracy: 0.5360, Loss: 1.0596
Epoch   0 Batch  189/538 - Train Accuracy: 0.4863, Validation Accuracy

Epoch   0 Batch  268/538 - Train Accuracy: 0.5456, Validation Accuracy: 0.5666, Loss: 0.7517
Epoch   0 Batch  269/538 - Train Accuracy: 0.5201, Validation Accuracy: 0.5646, Loss: 0.8227
Epoch   0 Batch  270/538 - Train Accuracy: 0.5160, Validation Accuracy: 0.5582, Loss: 0.8104
Epoch   0 Batch  271/538 - Train Accuracy: 0.5457, Validation Accuracy: 0.5517, Loss: 0.8001
Epoch   0 Batch  272/538 - Train Accuracy: 0.5125, Validation Accuracy: 0.5456, Loss: 0.8514
Epoch   0 Batch  273/538 - Train Accuracy: 0.5225, Validation Accuracy: 0.5384, Loss: 0.8206
Epoch   0 Batch  274/538 - Train Accuracy: 0.4771, Validation Accuracy: 0.5575, Loss: 0.8476
Epoch   0 Batch  275/538 - Train Accuracy: 0.5479, Validation Accuracy: 0.5648, Loss: 0.8325
Epoch   0 Batch  276/538 - Train Accuracy: 0.5486, Validation Accuracy: 0.5582, Loss: 0.7973
Epoch   0 Batch  277/538 - Train Accuracy: 0.5369, Validation Accuracy: 0.5442, Loss: 0.7999
Epoch   0 Batch  278/538 - Train Accuracy: 0.5441, Validation Accuracy

Epoch   0 Batch  357/538 - Train Accuracy: 0.5859, Validation Accuracy: 0.5755, Loss: 0.6610
Epoch   0 Batch  358/538 - Train Accuracy: 0.5482, Validation Accuracy: 0.5655, Loss: 0.6381
Epoch   0 Batch  359/538 - Train Accuracy: 0.6004, Validation Accuracy: 0.5710, Loss: 0.6280
Epoch   0 Batch  360/538 - Train Accuracy: 0.5686, Validation Accuracy: 0.5730, Loss: 0.6396
Epoch   0 Batch  361/538 - Train Accuracy: 0.5960, Validation Accuracy: 0.5753, Loss: 0.6127
Epoch   0 Batch  362/538 - Train Accuracy: 0.5631, Validation Accuracy: 0.5698, Loss: 0.6033
Epoch   0 Batch  363/538 - Train Accuracy: 0.5874, Validation Accuracy: 0.5694, Loss: 0.6126
Epoch   0 Batch  364/538 - Train Accuracy: 0.5561, Validation Accuracy: 0.5779, Loss: 0.6689
Epoch   0 Batch  365/538 - Train Accuracy: 0.5861, Validation Accuracy: 0.6101, Loss: 0.6309
Epoch   0 Batch  366/538 - Train Accuracy: 0.5686, Validation Accuracy: 0.6083, Loss: 0.6282
Epoch   0 Batch  367/538 - Train Accuracy: 0.5705, Validation Accuracy

...

Epoch   2 Batch  175/538 - Train Accuracy: 0.9324, Validation Accuracy: 0.9080, Loss: 0.0753
Epoch   2 Batch  176/538 - Train Accuracy: 0.9096, Validation Accuracy: 0.9080, Loss: 0.0912
Epoch   2 Batch  177/538 - Train Accuracy: 0.9301, Validation Accuracy: 0.9130, Loss: 0.0780
Epoch   2 Batch  178/538 - Train Accuracy: 0.9064, Validation Accuracy: 0.9151, Loss: 0.0737
Epoch   2 Batch  179/538 - Train Accuracy: 0.9311, Validation Accuracy: 0.9038, Loss: 0.0827
Epoch   2 Batch  180/538 - Train Accuracy: 0.9263, Validation Accuracy: 0.9029, Loss: 0.0800
Epoch   2 Batch  181/538 - Train Accuracy: 0.8992, Validation Accuracy: 0.9180, Loss: 0.0928
Epoch   2 Batch  182/538 - Train Accuracy: 0.9320, Validation Accuracy: 0.9187, Loss: 0.0719
Epoch   2 Batch  183/538 - Train Accuracy: 0.9407, Validation Accuracy: 0.9221, Loss: 0.0658
Epoch   2 Batch  184/538 - Train Accuracy: 0.9161, Validation Accuracy: 0.9208, Loss: 0.0845
Epoch   2 Batch  185/538 - Train Accuracy: 0.9430, Validation Accuracy

Epoch   2 Batch  264/538 - Train Accuracy: 0.9162, Validation Accuracy: 0.9130, Loss: 0.0721
Epoch   2 Batch  265/538 - Train Accuracy: 0.8953, Validation Accuracy: 0.9192, Loss: 0.0832
Epoch   2 Batch  266/538 - Train Accuracy: 0.9144, Validation Accuracy: 0.9167, Loss: 0.0726
Epoch   2 Batch  267/538 - Train Accuracy: 0.9270, Validation Accuracy: 0.9144, Loss: 0.0731
Epoch   2 Batch  268/538 - Train Accuracy: 0.9356, Validation Accuracy: 0.9144, Loss: 0.0551
Epoch   2 Batch  269/538 - Train Accuracy: 0.9297, Validation Accuracy: 0.9071, Loss: 0.0812
Epoch   2 Batch  270/538 - Train Accuracy: 0.9244, Validation Accuracy: 0.9077, Loss: 0.0691
Epoch   2 Batch  271/538 - Train Accuracy: 0.9270, Validation Accuracy: 0.9094, Loss: 0.0634
Epoch   2 Batch  272/538 - Train Accuracy: 0.9146, Validation Accuracy: 0.9105, Loss: 0.0813
Epoch   2 Batch  273/538 - Train Accuracy: 0.9121, Validation Accuracy: 0.9084, Loss: 0.0800
Epoch   2 Batch  274/538 - Train Accuracy: 0.8586, Validation Accuracy

Epoch   2 Batch  353/538 - Train Accuracy: 0.9055, Validation Accuracy: 0.9231, Loss: 0.0720
Epoch   2 Batch  354/538 - Train Accuracy: 0.9338, Validation Accuracy: 0.9228, Loss: 0.0697
Epoch   2 Batch  355/538 - Train Accuracy: 0.9404, Validation Accuracy: 0.9164, Loss: 0.0596
Epoch   2 Batch  356/538 - Train Accuracy: 0.9306, Validation Accuracy: 0.9148, Loss: 0.0574
Epoch   2 Batch  357/538 - Train Accuracy: 0.9322, Validation Accuracy: 0.9231, Loss: 0.0648
Epoch   2 Batch  358/538 - Train Accuracy: 0.9434, Validation Accuracy: 0.9283, Loss: 0.0601
Epoch   2 Batch  359/538 - Train Accuracy: 0.9334, Validation Accuracy: 0.9221, Loss: 0.0694
Epoch   2 Batch  360/538 - Train Accuracy: 0.9252, Validation Accuracy: 0.9240, Loss: 0.0668
Epoch   2 Batch  361/538 - Train Accuracy: 0.9280, Validation Accuracy: 0.9276, Loss: 0.0646
Epoch   2 Batch  362/538 - Train Accuracy: 0.9474, Validation Accuracy: 0.9272, Loss: 0.0592
Epoch   2 Batch  363/538 - Train Accuracy: 0.9163, Validation Accuracy

Epoch   2 Batch  442/538 - Train Accuracy: 0.9260, Validation Accuracy: 0.9142, Loss: 0.0524
Epoch   2 Batch  443/538 - Train Accuracy: 0.9144, Validation Accuracy: 0.9169, Loss: 0.0648
Epoch   2 Batch  444/538 - Train Accuracy: 0.9436, Validation Accuracy: 0.9270, Loss: 0.0586
Epoch   2 Batch  445/538 - Train Accuracy: 0.9346, Validation Accuracy: 0.9400, Loss: 0.0580
Epoch   2 Batch  446/538 - Train Accuracy: 0.9414, Validation Accuracy: 0.9423, Loss: 0.0573
Epoch   2 Batch  447/538 - Train Accuracy: 0.9158, Validation Accuracy: 0.9419, Loss: 0.0642
Epoch   2 Batch  448/538 - Train Accuracy: 0.9334, Validation Accuracy: 0.9418, Loss: 0.0568
Epoch   2 Batch  449/538 - Train Accuracy: 0.9332, Validation Accuracy: 0.9371, Loss: 0.0671
Epoch   2 Batch  450/538 - Train Accuracy: 0.9306, Validation Accuracy: 0.9355, Loss: 0.0786
Epoch   2 Batch  451/538 - Train Accuracy: 0.9057, Validation Accuracy: 0.9293, Loss: 0.0634
Epoch   2 Batch  452/538 - Train Accuracy: 0.9451, Validation Accuracy

Epoch   2 Batch  531/538 - Train Accuracy: 0.9266, Validation Accuracy: 0.9428, Loss: 0.0601
Epoch   2 Batch  532/538 - Train Accuracy: 0.9250, Validation Accuracy: 0.9286, Loss: 0.0524
Epoch   2 Batch  533/538 - Train Accuracy: 0.9328, Validation Accuracy: 0.9219, Loss: 0.0560
Epoch   2 Batch  534/538 - Train Accuracy: 0.9273, Validation Accuracy: 0.9219, Loss: 0.0499
Epoch   2 Batch  535/538 - Train Accuracy: 0.9472, Validation Accuracy: 0.9189, Loss: 0.0543
Epoch   2 Batch  536/538 - Train Accuracy: 0.9442, Validation Accuracy: 0.9205, Loss: 0.0662
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [50]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [51]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [52]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    
    text_ids = [vocab_to_int.get(word,vocab_to_int['<UNK>']) for word in sentence.lower().split()] 
    return text_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [53]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [47, 179, 211, 126, 62, 153, 63]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [244, 335, 335, 311, 89, 103, 227, 48, 259, 26, 196, 233, 237, 22]
  French Words: états-unis merveilleux merveilleux au printemps , et il est généralement agréable en mars .


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.